In [ ]:
#| default_exp formatting

In [ ]:
#| exporti 

from typing import (
    Union,
    Callable,
    TypeVar,
    Literal,
    List
)
from archetypon.base_model import *
from pydantic import root_validator
from pydantic.color import Color
import math
from pandas import isnull
import pandas as pd
import datetime as dt

## String Formatting

### Format Factory

In [ ]:
#| exporti 

FormatterT = TypeVar("FormatterT",str,Callable)

In [ ]:
#| export

def format_factory(
    string_or_callable:FormatterT,
    null_format:str = '',
    **kwargs
)->Callable:
    """
    Returns a function factory to format a given value. 
    """
    if callable(string_or_callable):
        def _formatter(val,*args):

            if val=='' or isnull(val):
                return null_format
            return string_or_callable(val,*args,**kwargs)
        
        return _formatter
    
    def _formatter(val):
        if val=='' or isnull(val):
            return null_format

        return string_or_callable.format(val)
    return _formatter

In [ ]:
commas = format_factory("{:,.0f}")
assert commas(1234.0) == '1,234'

In [ ]:
def convert_snake_case(string:str,title:bool=False)->str:
    formatted = " ".join(x for x in string.split('_'))
    if title:
        return formatted.title()
    return formatted

snake_to_title = format_factory(convert_snake_case,title=True)
assert snake_to_title("snake_case") == "Snake Case"

### Formatting Functions

In [ ]:
#| export

def big_number(num:float,decimal_places:int=2)->str:
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    formatted = f'%.{decimal_places}f%s' % (num, ['', 'K', 'M', 'B', 'T', 'P'][magnitude])
    return formatted

In [ ]:
#| export

def big_dollars(num:float,decimal_places:int=2)->str:
    formatted = big_number(num,decimal_places)
    return f"${formatted}"

In [ ]:
#| export

def as_multiple(num:float)->str:
    if abs(num) < 0.005:
        return '-'
    else:
        return '{:.2f}x'.format(num)

In [ ]:
assert big_number(1e6) == '1.00M'
assert big_number(1234567)=='1.23M'

assert big_dollars(1e6)=='$1.00M'
assert big_dollars(1234567891) == '$1.23B'

assert as_multiple(1.2) == "1.20x"

In [ ]:
#| export 

def format_minutes(time:float)->str:
    """Takes in minutes as a float and converts to MM:SS"""
    if type(time)==dt.time:
        return time.strftime("%M:%S")
    minutes = math.floor(time)
    seconds = (time * 60) % 60
    return "%02d:%02d" % (minutes, seconds)

### Formatter

In [ ]:
#| export

class Formatter():
    """
    A customizable object for applying string formats. 
    
    Any additional attributes passed during instantion must have a name ending in "_format". 
    Values can be either a formattable string (e.g. "{:.0f}") or a callable.  
    """
    def __init__(
        self,
        null_format: str = '',
        dollars_format: FormatterT = "${:,.0f}",
        percent_format: FormatterT = "{:.0%}",
        percent2dp_format: FormatterT = "{:.2%}",
        number_format: FormatterT = "{:,.0f}",
        small_number_format: FormatterT = "{:.2f}",
        big_number_format: FormatterT = big_number,
        big_dollars_format: FormatterT = big_dollars,
        multiple_format: FormatterT = as_multiple,
        minutes_format: FormatterT = format_minutes,
        **kwargs
    ):
        self.null_format = null_format
        self.dollars_format=dollars_format
        self.percent_format=percent_format
        self.percent2dp_format=percent2dp_format
        self.number_format=number_format
        self.small_number_format=small_number_format
        self.big_number_format=big_number_format
        self.big_dollars_format=big_dollars_format
        self.multiple_format = multiple_format
        self.minutes_format = minutes_format
        
        #add anything extra
        for k,v in kwargs.items():
            if not k.split('_')[-1]=='format':
                raise ValueError(f"Keyword arguments passed to the Formatter must end in '_format' ")
            setattr(self,k,v)
        
        # create methods from string_formats
        items = [(k,v) for k,v in self.__dict__.items()]
        for k,v in items:
            if not k.startswith('null'):
                method = '_'.join([x for x in k.split('_')][:-1]) # remove "_format" from the end of the attribute when setting the method
                if type(v)==tuple:
                    setattr(
                        self,
                        method,
                        format_factory(
                            v[0],
                            null_format = self.null_format,
                            **v[1]
                        )
                    )
                else:
                    setattr(
                        self,
                        method,
                        format_factory(v,null_format=self.null_format)
                    )

#### Example

In [ ]:
fmt = Formatter(
    # adding custom formats
    date_format = lambda x: pd.Timestamp(x).strftime('%b %d %Y'),
    month_format = "{:%b %Y}",
    millions_format = lambda x: "{:.0f}mm".format(x/1e6),
)



assert fmt.big_dollars(1234567) == big_dollars(1234567) == '$1.23M'
assert fmt.multiple(1.22999)=='1.23x'
assert fmt.percent(.0123)=="1%"
assert fmt.percent2dp(.0123)=="1.23%"
assert fmt.date('20220212')=='Feb 12 2022'
assert fmt.month(dt.date(1994,6,11))=='Jun 1994'
assert fmt.minutes(60.5)=="60:30"
assert fmt.millions(123456789) == "123mm"

#### Formatting Null Values

The `null_format` attribute of the Formatter object is a string that gets returned when any method is called on a null value.  

By default, nulls are represented as empty strings

In [ ]:
import numpy as np

In [ ]:
assert fmt.big_dollars(np.nan)==fmt.number(np.nan)==''==fmt.null_format

Formatter will treat empty strings as null.

In [ ]:
assert fmt.percent('')==fmt.null_format

In [ ]:
custom_null_format = "I'm a null value!"
fmt_custom_null = Formatter(null_format=custom_null_format)
assert fmt_custom_null.big_number(np.nan)==fmt_custom_null.number(np.nan)==custom_null_format

Override default implementation at instantiation: 

In [ ]:
fmt_custom_dollars = Formatter(
    dollars_format="${:,.2f}"
)
assert fmt_custom_dollars.dollars(12345.678) == "$12,345.68"

In [ ]:
fmt_custom = Formatter(
    snake_format = convert_snake_case
)
fmt_custom.snake("snake_case")

'snake case'

In [ ]:
# if you need to pass arguments to the function factory 
fmt_custom_with_kwargs = Formatter(
    snake_to_camel_format = (
        convert_snake_case,{'title':True}
    ),
    big_dollars_format = (
        big_dollars,
        {'decimal_places':4}
    )
)
assert fmt_custom_with_kwargs.snake_to_camel("snake_case")=='Snake Case'

In [ ]:
assert fmt_custom.big_dollars(1.234e9)=='$1.23B'
assert fmt_custom_with_kwargs.big_dollars(1.1234e9)=='$1.1234B'

In [ ]:
#| include: false
!nbdev_export